In [4]:
import sys
sys.path.append("../")

import matplotlib.pyplot as plt
import numpy as np
from src.loaders import load_data, load_labels
from src.classify import MCM_Classifier
from src.plot import  plot_results
import os
from src.plot import plot_confusion_matrix, plot_label_prob_diff
import torch

ModuleNotFoundError: No module named 'torch'

In [3]:
# Customizable environment variables
n_categories = 10  # Number of categories to be classified
n_variables = 121  # Number of variables in the dataset
mcm_filename_format = "train-images-unlabeled-{}_comms.dat"
data_filename_format = "train-images-unlabeled-{}.dat"
data_path = "../INPUT/data/"
communities_path = "../INPUT/MCMs/"
output_path = "../OUTPUT/"

In [7]:
size = 2**121
res = np.zeros(size)

ValueError: Maximum allowed dimension exceeded

In [12]:
import torch

size = (2**12,)
indices = []
values = []
tensor = torch.sparse_coo_tensor(indices, values, size=size)


RuntimeError: number of dimensions must be sparse_dim (0) + dense_dim (0), but got 1

In [29]:
import scipy
A = scipy.sparse.bsr_array((1, 2**121))
A


OverflowError: Python int too large to convert to C long

In [34]:
indices = torch.LongTensor([1,3,4,6])
values = torch.FloatTensor([1,1,1,1])
size = (2**121,)
t = torch.sparse_coo_tensor(indices.unsqueeze(0), values, size=size)
t

TypeError: sparse_coo_tensor(): argument 'size' failed to unpack the object at pos 1 with error "Overflow when unpacking long"